# pure regression

In [ ]:
import time
import os
import gc
import pandas as pd
import numpy as np
from sklearn import linear_model

input_dir = os.path.join(os.pardir, 'input')
print('Loading data...')
train_df = pd.read_csv(os.path.join(input_dir, 'sales_train.csv'))
test_df = pd.read_csv(os.path.join(input_dir, 'test.csv'))
item_df = pd.read_csv(os.path.join(input_dir, 'items.csv'))
print(' Train data size is %d x %d'%(train_df.shape[0],train_df.shape[1]))
#adding the testdata that were omitted...
#adding the testdata and fill in some empty fields
test_df['date_block_num']=0
test_df['item_cnt_day']=0.285  # found in a script this is an optimal input

# reverse data block
train_df['date_block_num'] = 33 - train_df['date_block_num'] # change it, so that it is 0 to 9, reversing order of monthblock
# estimate days since last sales
train_df['date'] = pd.to_datetime(train_df.date,format="%d.%m.%Y")
itemrange= pd.DataFrame( (train_df.groupby('item_id').date.max()-train_df.groupby('item_id').date.min() ).dt.days ) #-train_df.date.min()

# append Test data
train_df=train_df.append(test_df.drop('ID',axis=1),sort=False )#,left_index=True,right_index=True)
# add category
train_df=train_df.merge(item_df,how='left',left_on='item_id',right_on='item_id')
train_df=train_df.merge(itemrange,how='left',left_index=True,right_index=True)

print(train_df.shape,
      train_df.head())



# adding shop.item data 

essentially
generating pivot and using all information available again
and svd the principal axis

In [ ]:
def piv_clust(data,veld,kolom,waarde,komponent):
    from sklearn.decomposition import TruncatedSVD,FastICA
    df = data.pivot_table(index=veld, columns=kolom, values=waarde, fill_value=0, aggfunc=np.sum)            #pivot table > signal should follow
    svd = TruncatedSVD(n_components=komponent, n_iter=7, random_state=42)                                    #decomp functions
    ica = FastICA(n_components=komponent)
    df_norm =( (df - df.mean()) / (df.max() - df.min()) ).fillna(0)                                          #normalize
    return pd.DataFrame( np.concatenate([svd.fit_transform(df_norm)*svd.singular_values_, ica.fit_transform(df_norm)],axis=1) , index=df.index),svd.explained_variance_ratio_  #U*S

train_p = train_df.pivot_table(index=['shop_id', 'item_id'], columns='date_block_num', values='item_cnt_day', fill_value=0, aggfunc=np.sum)
print(train_p.shape,train_df.head())

train_i,sing_i=piv_clust(train_df,'item_id',['shop_id','date_block_num'],'item_price',10)
train_s,sing_s=piv_clust(train_df,'shop_id',['item_category_id','date_block_num'],'item_price',10)
train_si,sing_si=piv_clust(train_df[train_df['date_block_num']>10],'item_id',['date_y'],'item_cnt_day',10)
train_si



# optimal clipping  25-28
so instead of clipping at 20, we have some headroom


def piv_clust(data,veld,kolom,waarde,komponent):
    from sklearn.decomposition import TruncatedSVD
    df = data.pivot_table(index=veld, columns=kolom, values=waarde, fill_value=0, aggfunc=np.max)
    svd = TruncatedSVD(n_components=komponent, n_iter=7, random_state=42)
    df_norm =( (df - df.mean()) / (df.max() - df.min()) ).fillna(0)
    return pd.DataFrame( svd.fit_transform(df_norm), index=df.index)*svd.singular_values_,svd.explained_variance_ratio_  #U*S


train_si,sing_si=piv_clust(train_df,'item_id',['shop_id', 'date_block_num'],'tijdratio',10)

In [ ]:
def submfi(tr_te,test,pre,file):
                                          #pre is prediction, 
                                          #test is the file to submit, 
                                          #tr_te is the training file used to predict
    tr_te['pre']=pre
    s_df=tr_te[['pre']].clip(0)           #clipp all negative values
    s_df=s_df.merge(test,how='right',left_index=True,right_on=['shop_id','item_id']) #find all records linked to test
    s_df=s_df[['pre','ID']]               #select columns to submit
    s_df.columns=['item_cnt_month','ID']  #rename columns
    s_df.to_csv(file, index=False)        #save file 
    return

In [ ]:
# Create linear regression object
regr = linear_model.LinearRegression()

for xi in range(26,27):
    # Train the model
    x = train_p.iloc[:,1:].clip(0,xi)
    y = train_p.iloc[:,0].clip(0,20)
    regr.fit(x, y)
    # Make predictions
    pred = regr.predict(x)
    # The coefficients
    print('Intercept: \n', regr.intercept_)
    print('Coefficients: \n', regr.coef_)
    # The mean squared error
    print(xi,'MSRE: %.4f'%np.sqrt(((y-pred)*(y-pred)).mean()))
    
p=regr.predict(train_p.iloc[:,0:-1].clip(0,xi).fillna(0))
submfi(train_p,test_df,p,'subm0.csv')    

# merging  the extra clustered info

In [ ]:
train_t=train_p.clip(-1,26).merge(train_i,how='left',left_index=True,right_index=True)    # 0.6436
train_t=train_t.merge(train_s,how='left',left_index=True,right_index=True)     # 0.6621 0.6419
train_t=train_t.merge(train_si,how='left',left_index=True,right_index=True)   #0.6551 0.6533 0.6361
print(train_t.shape)
for xi in range(25,26):
    x = train_t.iloc[:,1:].fillna(0)
    y = train_t.iloc[:,0].clip(0,20)
    regr.fit(x, y)
    # Make predictions
    pred = regr.predict(x)
    # The coefficients
    print('Intercept: \n', regr.intercept_)
    print('Coefficients: \n', regr.coef_)
    # The mean squared error
    print(xi,'MSRE: %.4f'%np.sqrt(((y-pred)*(y-pred)).mean()))
    
# prepare submission
t_ = train_p.iloc[:,0:-1].clip(-1,26)
t_=t_.merge(train_i,how='left',left_index=True,right_index=True)
t_=t_.merge(train_s,how='left',left_index=True,right_index=True)
t_=t_.merge(train_si,how='left',left_index=True,right_index=True)
print(t_.shape)


p=regr.predict(t_.fillna(0))
submfi(train_p,test_df,p,'subm1.csv')

# create triple exp data grid
create moving average / moving average diff / first difference / second difference

In [ ]:
def prepar(data):
    X=data.T
    lengte=data.shape[1]
    print(data.shape,lengte, ['A'+str(x) for x in range(lengte)] +['Ad'+str(x) for x in range(1,lengte)]+['Add'+str(x) for x in range(2,lengte)]+['Am'+str(x) for x in range(1,lengte)]+['Amd'+str(x) for x in range(2,lengte)] )
    Xd=X.diff().dropna()
    Xdd=Xd.diff().dropna()
    Xm=X.rolling(2).mean().dropna()
    Xmd=Xm.diff().dropna()
    Xt=((((X.append(Xd)).append(Xdd)).append(Xm)).append(Xmd)).T
    
    Xt.columns=['A'+str(x) for x in range(lengte)] +['Ad'+str(x) for x in range(1,lengte)]+['Add'+str(x) for x in range(2,lengte)]+['Am'+str(x) for x in range(1,lengte)]+['Amd'+str(x) for x in range(2,lengte)]  #['A1', 'A2', 'A3','A4', 'Ad2', 'Ad3','Ad4', 'Add3','Add4', 'Am2', 'Am3','Am4', 'Amd3','Amd4']
    return Xt

#prepar(train_p.iloc[:,:])

# optimal number of months added = 6

In [ ]:
x = prepar(train_p.iloc[:,1:11])
y = train_p.iloc[:,0]
for yi in range(2,7):
    x=x.append( prepar(train_p.iloc[:,yi:yi+10]) )
    y=y.append( train_p.iloc[:,yi-1] )

#x=x.merge(train_i,left_index=True,right_index=True)
#x=x.merge(train_s,left_index=True,right_index=True)
#x=x.merge(train_si,left_index=True,right_index=True)
print(x.shape,y.shape)

for xi in range(25,26):
    x = x.clip(-1,26)
    y = y.clip(0,20)
    regr.fit(x, y)
    # Make predictions
    pred = regr.predict(x)
    # The coefficients
    print('Intercept: \n', regr.intercept_)
    print('Coefficients: \n', regr.coef_)
    # The mean squared error
    print(xi,'MSRE: %.4f'%np.sqrt(((y-pred)*(y-pred)).mean()))

In [ ]:
x_m=x

# merging with clusters

In [ ]:
#x_m  msre 0.5775

x_=x_m.merge(train_i,left_index=True,right_index=True)    # item / price .5632
print(x_m.shape,x_.shape,y.shape)
x_=x_.merge(train_s,left_index=True,right_index=True)   # shop/count .5651  .5629
x_=x_.merge(train_si,how='left',left_index=True,right_index=True)   #item / sumtimeratio 0.5654
print(x_m.shape,x_.shape)

for xi in range(25,26):
    x = x_.clip(-1,26).fillna(0)
    y = y.clip(0,20)
    regr.fit(x, y)
    # Make predictions
    pred = regr.predict(x)
    # The coefficients
    print('Intercept: \n', regr.intercept_)
    print('Coefficients: \n', regr.coef_)
    # The mean squared error
    print(xi,'MSRE: %.4f'%np.sqrt(((y-pred)*(y-pred)).mean()))

In [ ]:
# prepare submission
t_ = prepar(train_p.iloc[:,0:10])
t_=t_.merge(train_i,left_index=True,right_index=True)
t_=t_.merge(train_s,left_index=True,right_index=True)
t_=t_.merge(train_si,how='left',left_index=True,right_index=True)


p=regr.predict(t_.clip(-1,26).fillna(0))
print(p)

submfi(train_p,test_df,p,'subm2.csv')

train_p['pre']=p
s_df=train_p[['pre']].clip(0)
s_df=s_df.merge(test_df,how='right',left_index=True,right_on=['shop_id','item_id'])
s_df=s_df[['pre','ID']]
s_df.columns=['item_cnt_month','ID']
s_df.to_csv('submission.csv', index=False)


#=test_df.merge()
